In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
# import re
import google.generativeai as genai


In [ ]:
web = webdriver.Chrome()
web.get('https://forms.gle/nm5JjrN6BHiiKFjVA')
time.sleep(5)

In [ ]:
genai.configure(api_key="MY_API_KEY")
generation_config = {"temperature": 0.9, "top_p": 1, "top_k": 1, "max_output_tokens": 2048}
model = genai.GenerativeModel("gemini-pro", generation_config = generation_config)

In [ ]:
## Auto Google form filler

def fill_form(sname,company,date,head,con,sum,radio):
    
    global names
    global namec
    global named
    global nameh
    global cname
    global namesum
    

    names = web.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[1]/div/div/div[2]/div/div[1]/div/div[1]/input')
    names.send_keys(sname)

    namec = web.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/div[1]/div/div[1]/input')
    namec.send_keys(company)

    named = web.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[3]/div/div/div[2]/div/div/div[2]/div[1]/div/div[1]/input')
    named.send_keys(date)

    nameh = web.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[4]/div/div/div[2]/div/div[1]/div/div[1]/input')
    nameh.send_keys(head)

    cname = web.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[5]/div/div/div[2]/div/div[1]/div[2]/textarea')
    cname.send_keys(con)

    namesum = web.find_element('xpath', '//*[@id="mG61Hd"]/div[2]/div/div[2]/div[6]/div/div/div[2]/div/div[1]/div[2]/textarea')
    namesum.send_keys(sum)

    # time.sleep(2)
    button = web.find_elements(By.CLASS_NAME, "Od2TWd")
    for rad in button:
        # if rad.get_attribute('data-value') == "Politics":
        #     rad.click()
        if rad.get_attribute('data-value') == radio:
            rad.click()
    # time.sleep(3)
    
    try:
        namesubmit = web.find_element('xpath','//*[@id="mG61Hd"]/div[2]/div/div[3]/div[1]/div[1]/div/span/span')
        namesubmit.click()

        time.sleep(1)
        another_one = web.find_element('xpath','/html/body/div[1]/div[2]/div[1]/div/div[4]/a')
        another_one.click()
        time.sleep(1)
        # names.clear()
        # namec.clear()
        # named.clear()
        # nameh.clear()
        # cname.clear()
        # namesum.clear()

    except:
        print(head)
        names.clear()
        namec.clear()
        named.clear()
        nameh.clear()
        cname.clear()
        namesum.clear()

In [ ]:
url = 'https://timesofindia.indiatimes.com/2013/5/24/archivelist/year-2013,month-5,starttime-41418.cms'

In [ ]:
r = requests.get(url)
html_content = r.text
soup = BeautifulSoup(html_content, 'lxml')  
links = [a.get('href') for a in soup.find_all('a', href=True)]
print(links)

In [ ]:
urls = [k for k in links if 'http://timesofindia.indiatimes.com' in k]

In [ ]:
len(urls)

In [ ]:
done = []

In [ ]:
new = [x for x in urls if x not in done]
len(new)
len(done)
## News Extraction

i = 0
for url in new:
    # i+=1
    done.append(url)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('h1').text
    # try:
    #     title = soup.find('h1', class_='HNMDR').text
    # except:
    #     continue
    # date = soup.find('div', class_='xf8Pm byline').text
    # date = re.search(r'\w+ \d{1,2}, \d{4}', date).group(0) 
    text = soup.find('div',class_='_s30J clearfix').text
    # text = ''
    # for para in soup.find_all('p'):
    #     text += para.text + '\n\n'

    ## Summarization using Gemini (Bard)

    time.sleep
    try:
        response = model.generate_content("Summarize the following text in 100 words: "+ text)
        context = response.text
        # print('resdone')
    except:
        context = ""
        print(title)
        continue
    
    try:
        newscat = model.generate_content("Categorize this news into just one (only one) of the most appropriate field representing the news ['Politics','International News','National News','Local News','Politics','Business and Finance','Science and Technology','Health and Wellness','Entertainment','Sports','Lifestyle and Features','Opinion and Editorial','Environment','Education','Crime and Justice','Human Interest', 'Obituaries','Weather','Religion and Spirituality','Technology and Gadgets','Automotive']: "+ text)
        newscat= newscat.text
        # print('newdone')

    except:
        newscat = "" 
        print(title)
        continue

    sname = 'Mann Kurani'
    company = 'The Times of India'
    date = '20/07/2022'
    head= title
    con= text
    sum= context
    radio = newscat
    # print(radio)
    fill_form(sname,company,date,head,con,sum,radio)
    
    i+=1
    print(i)
